In [15]:
pip install langchain_groq


In [16]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    groq_api_key='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',
    model_name="llama-3.1-70b-versatile"
)

response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


In [17]:
pip install langchain


In [18]:
pip install langchain_community

In [19]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-33460")
page_data = loader.load().pop().page_content
print(page_data)



Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsGIFT CARDSPROMOTIONSFIND A STORESIGN UP FOR EMAILBECOME A MEMBERNIKE JOURNALSEND US FEEDBACKGET HELPGET HELPOrder StatusShipping and Del

In [20]:
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate

# Initialize the ChatGroq object
llm = ChatGroq(
    temperature=0,
    groq_api_key='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',  # Use your valid API key here
    model_name="llama-3.1-70b-versatile"
)

# Define the prompt template for extraction
prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the career's page of a website.
    Your job is to extract the job postings and return them in JSON format containing the
    following keys: `role`, `experience`, `skills` and `description`.
    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLE):
    """
)

# Create the extraction chain by combining the prompt template with the LLM (llm)
chain_extract = prompt_extract | llm

# Example usage: invoke the chain with actual page data
res = chain_extract.invoke(input={'page_data': page_data})

# Type check
print(type(res.content))

# Further processing of the extracted result if necessary


<class 'str'>


In [21]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'Software Engineer',
  'experience': '3-5 years',
  'skills': ['Java', 'Python', 'Cloud Computing'],
  'description': 'Design and develop software applications for our e-commerce platform.'},
 {'role': 'Data Analyst',
  'experience': '1-3 years',
  'skills': ['Excel', 'SQL', 'Data Visualization'],
  'description': 'Analyze sales data and provide insights to inform business decisions.'},
 {'role': 'Marketing Manager',
  'experience': '5-7 years',
  'skills': ['Marketing Strategy', 'Brand Management', 'Social Media'],
  'description': 'Develop and execute marketing campaigns to promote our products.'}]

In [22]:
type(json_res)

list

In [23]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [24]:
pip install chromadb


In [25]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [26]:
# Assuming json_res is a list of dictionaries
json_res = [
    {'skills': ['Python', 'Machine Learning']},
    {'location': 'New York'},
    {'experience': 5}
]

# Create an empty dictionary
combined_dict = {}

# Loop through each dictionary in the list and update the combined dictionary
for item in json_res:
    combined_dict.update(item)

# Now you can access 'skills' without errors
skills = combined_dict.get('skills', [])
print("Skills:", skills)

# Example of querying links if you have a collection object
# Assuming 'collection' is defined and set up properly
if skills:  # Check if skills list is not empty
    links = collection.query(query_texts=skills, n_results=2).get('metadatas', [])
    print("Links:", links)
else:
    print("No skills found.")


Skills: ['Python', 'Machine Learning']
Links: [[{'links': 'https://example.com/ml-python-portfolio'}, {'links': 'https://example.com/python-portfolio'}], [{'links': 'https://example.com/ml-python-portfolio'}, {'links': 'https://example.com/wordpress-portfolio'}]]


In [28]:
job = json_res
job['skills']

TypeError: list indices must be integers or slices, not str

In [29]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}

        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools.
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability,
        process optimization, cost reduction, and heightened overall efficiency.
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ.
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):

        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Python and Machine Learning Solutions for Your Business

Dear Hiring Manager,

I came across your job description requiring a professional with expertise in Python and Machine Learning, and I am excited to introduce you to AtliQ, a leading AI & Software Consulting company. With our extensive experience in developing tailored solutions, we are confident that we can fulfill your needs and take your business to the next level.

At AtliQ, we have a proven track record of empowering enterprises with scalable, optimized, and cost-effective solutions. Our team of experts has a deep understanding of Python and Machine Learning, and we have successfully implemented numerous projects that showcase our capabilities.

I would like to bring to your attention some of our notable projects that demonstrate our expertise in Python and Machine Learning:

- https://example.com/ml-python-portfolio (Machine Learning and Python Portfolio)
- https://example.com/python-portfolio (Python Portfo